In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用Claude 3模型快速入门
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/generative_ai/anthropic_claude_3_intro.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在Colab中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fblob%2Fmain%2Fnotebooks%2Fofficial%2Fgenerative_ai%2Fanthropic_claude_3_intro.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> 在Colab Enterprise中打开
    </a>
  </td> 
  <td style="text-align: center">                                                                             
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/generative_ai/anthropic_claude_3_intro.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在Workbench中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/generative_ai/anthropic_claude_3_intro.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在GitHub上查看
    </a>
  </td>
  
</table>

## 概述

### Vertex AI 上的Claude

Anthropic在Vertex AI上提供的Claude 3模型是完全托管的无服务器模型，可作为托管API提供。要在Vertex AI上使用Claude模型，请直接向Vertex AI API端点发送请求。

您可以将Claude响应流式传输，以减少最终用户的延迟感知。流式响应使用服务器发送事件（SSE）逐步传输响应。

### 可用的Anthropic Claude模型

#### Claude 3.5 Sonnet
Anthropic最强大的AI模型。Claude 3.5 Sonnet在速度较高且成本较低的情况下，优于竞争对手的模型和Claude 3 Opus。

#### Claude 3 Opus
Claude 3 Opus是Anthropic第二智能的AI模型，在高度复杂任务上具有顶级性能。

#### Claude 3 Haiku
Anthropic Claude 3 Haiku是Anthropic最快速、最紧凑的视觉和文本模型，可实现对简单查询的几乎即时响应，旨在无缝模拟人类交互的AI体验。

#### Claude 3 Sonnet
Anthropic Claude 3 Sonnet经过工程处理，可在各种用例中进行可靠的规模化AI部署。

所有Claude 3模型都可以处理图像并返回文本输出，并具有200K上下文窗口。

## 目标

本笔记本展示如何使用**Vertex AI API**和**Anthropic的Python Vertex SDK**调用Vertex AI上的Claude API，使用Claude 3 Sonnet和Claude 3 Haiku模型。

有关更多信息，请参阅[使用Claude](https://cloud.google.com/vertex-ai/generative-ai/docs/partner-models/use-claude)文档。

## 顶点 AI API

开始吧

安装所需的软件包

In [ ]:
! pip3 install -U -q httpx

重新启动运行时（仅限Colab）

为了使用新安装的软件包，您必须重新启动Google Colab上的运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️内核将重新启动。在继续下一步之前，请等待它完成。⚠️</b>
</div>

### 在笔记本环境上进行身份验证（仅限Colab）

在Google Colab上对您的环境进行身份验证。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

选择克劳德3型号

In [ ]:
MODEL = "claude-3-5-sonnet@20240620"  # @param ["claude-3-5-sonnet@20240620", "claude-3-opus@20240229", "claude-3-haiku@20240307", "claude-3-sonnet@20240229" ]
if MODEL == "claude-3-5-sonnet@20240620":
    available_regions = ["us-east5", "europe-west1"]
elif MODEL == "claude-3-opus@20240229":
    available_regions = ["us-east5"]
elif MODEL == "claude-3-haiku@20240307":
    available_regions = ["us-east5", "europe-west1"]
elif MODEL == "claude-3-sonnet@20240229":
    available_regions = ["us-east5"]

选择一个位置

In [ ]:
import ipywidgets as widgets
from IPython.display import display

dropdown = widgets.Dropdown(
    options=available_regions,
    description="Select a location:",
    font_weight="bold",
    style={"description_width": "initial"},
)


def dropdown_eventhandler(change):
    global LOCATION
    if change["type"] == "change" and change["name"] == "value":
        LOCATION = change.new
        print("Selected:", change.new)


LOCATION = dropdown.value
dropdown.observe(dropdown_eventhandler, names="value")
display(dropdown)

设置 Google 云项目和模型信息

要开始使用 Vertex AI，您必须拥有现有的 Google 云项目并[启用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。了解更多关于[设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)的信息。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
ENDPOINT = f"https://{LOCATION}-aiplatform.googleapis.com"

if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    raise ValueError("Please set your PROJECT_ID")

#### 导入所需的库

In [ ]:
import base64
import json

import httpx
import requests
from IPython.display import Image

文本生成

一元调用

向指定的API端点发送一个POST请求，使用提供的载荷来获取香蕉面包食谱模型的响应。

In [ ]:
PAYLOAD = {
    "anthropic_version": "vertex-2023-10-16",
    "messages": [{"role": "user", "content": "Send me a recipe for banana bread."}],
    "max_tokens": 100,
    "stream": False,
}

request = json.dumps(PAYLOAD)
!curl -X POST -H "Authorization: Bearer $(gcloud auth print-access-token)" -H "Content-Type: application/json" {ENDPOINT}/v1/projects/{PROJECT_ID}/locations/{LOCATION}/publishers/anthropic/models/{MODEL}:rawPredict -d '{request}'

#### 流式调用

向指定的API端点发送一个POST请求，使用提供的有效荷载从模型中流式传输香蕉面包食谱的响应。

In [ ]:
PAYLOAD = {
    "anthropic_version": "vertex-2023-10-16",
    "messages": [{"role": "user", "content": "Send me a recipe for banana bread."}],
    "max_tokens": 100,
    "stream": True,
}

request = json.dumps(PAYLOAD)
!curl -X POST -H "Authorization: Bearer $(gcloud auth print-access-token)" -H "Content-Type: application/json" {ENDPOINT}/v1/projects/{PROJECT_ID}/locations/{LOCATION}/publishers/anthropic/models/{MODEL}:streamRawPredict -d '{request}'

### 视野

编码和预览图片

我们使用httpx库从维基百科获取样本图片，但您可以使用任何适合您的图片来源。

In [ ]:
image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/a/a7/Camponotus_flavomarginatus_ant.jpg/300px-Camponotus_flavomarginatus_ant.jpg"
image_b64 = base64.b64encode(httpx.get(image_url).content).decode("utf-8")

response = requests.get(image_url)
image = Image(response.content, width=300, height=200)

image

单次调用

将POST请求发送到指定的API端点，以获取模型分析图像内容的响应，提供基于base64编码的数据，以及文本提示。

In [ ]:
PAYLOAD = {
    "anthropic_version": "vertex-2023-10-16",
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": "image/jpeg",
                        "data": image_b64,
                    },
                },
                {"type": "text", "text": "What is in this image?"},
            ],
        }
    ],
    "max_tokens": 100,
    "stream": False,
}

request = json.dumps(PAYLOAD)
!curl -X POST -H "Authorization: Bearer $(gcloud auth print-access-token)" -H "Content-Type: application/json" {ENDPOINT}/v1/projects/{PROJECT_ID}/locations/{LOCATION}/publishers/anthropic/models/{MODEL}:rawPredict -d '{request}'

#### 实时调用

向指定的 API 端点发送 POST 请求，以从分析图像内容的模型流式传输响应，同时提供以 base64 编码的数据和文本提示。

In [ ]:
PAYLOAD = {
    "anthropic_version": "vertex-2023-10-16",
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": "image/jpeg",
                        "data": image_b64,
                    },
                },
                {"type": "text", "text": "What is in this image?"},
            ],
        }
    ],
    "max_tokens": 100,
    "stream": True,
}

request = json.dumps(PAYLOAD)
!curl -X POST -H "Authorization: Bearer $(gcloud auth print-access-token)" -H "Content-Type: application/json" {ENDPOINT}/v1/projects/{PROJECT_ID}/locations/{LOCATION}/publishers/anthropic/models/{MODEL}:streamRawPredict -d '{request}'

使用Anthropic的顶点SDK进行Python编程

开始吧

### 安装 Anthropic 的 Vertex SDK for Python 和其他所需的包

In [ ]:
! pip3 install -U -q 'anthropic[vertex]'
! pip3 install -U -q httpx

重新启动运行时（仅限Colab）

要使用新安装的软件包，您必须重新启动Google Colab上的运行时。

In [ ]:
# Restart kernel after installs so that your environment can access the new packages
import sys

if "google.colab" in sys.modules:
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️内核将重新启动。在继续下一步之前，请等待其完成。⚠️</b>
</div>

### 验证您的笔记本环境（仅限Colab）

在Google Colab上验证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

选择Claude 3型号

In [ ]:
MODEL = "claude-3-5-sonnet@20240620"  # @param ["claude-3-5-sonnet@20240620", "claude-3-opus@20240229", "claude-3-haiku@20240307", "claude-3-sonnet@20240229" ]
if MODEL == "claude-3-5-sonnet@20240620":
    available_regions = ["us-east5", "europe-west1"]
elif MODEL == "claude-3-opus@20240229":
    available_regions = ["us-east5"]
elif MODEL == "claude-3-haiku@20240307":
    available_regions = ["us-east5", "europe-west1"]
elif MODEL == "claude-3-sonnet@20240229":
    available_regions = ["us-east5"]

选择一个地区

In [ ]:
import ipywidgets as widgets
from IPython.display import display

dropdown = widgets.Dropdown(
    options=available_regions,
    description="Select a location:",
    font_weight="bold",
    style={"description_width": "initial"},
)


def dropdown_eventhandler(change):
    global LOCATION
    if change["type"] == "change" and change["name"] == "value":
        LOCATION = change.new
        print("Selected:", change.new)


LOCATION = dropdown.value
dropdown.observe(dropdown_eventhandler, names="value")
display(dropdown)

#### 设置Google Cloud项目和模型信息

要开始使用Vertex AI，您必须拥有一个现有的Google Cloud项目并[启用Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。了解更多关于[设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)的信息。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
ENDPOINT = f"https://{LOCATION}-aiplatform.googleapis.com"

if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    raise ValueError("Please set your PROJECT_ID")

导入所需的库

In [ ]:
import base64

import httpx
import requests
from IPython.display import Image

文本生成

一元调用

初始化一个用于Anthropic的Vertex AI的客户端，发送一个请求来生成内容，并以格式化的JSON格式打印响应。

In [ ]:
from anthropic import AnthropicVertex

client = AnthropicVertex(region=LOCATION, project_id=PROJECT_ID)
message = client.messages.create(
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": "Send me a recipe for banana bread.",
        }
    ],
    model=MODEL,
)
print(message.model_dump_json(indent=2))

流式调用

初始化Anthropic的Vertex AI客户端，发送流式请求以生成内容，并持续打印接收到的文本。

In [ ]:
from anthropic import AnthropicVertex

client = AnthropicVertex(region=LOCATION, project_id=PROJECT_ID)

with client.messages.stream(
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": "Send me a recipe for banana bread.",
        }
    ],
    model=MODEL,
) as stream:
    for text in stream.text_stream:
        print(text, end="", flush=True)

### 愿景

#### 编码和预览图片

我们使用httpx库从维基百科获取示例图片，但您可以使用任何适合您的图片来源。

In [ ]:
image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/a/a7/Camponotus_flavomarginatus_ant.jpg/300px-Camponotus_flavomarginatus_ant.jpg"
image_media_type = "image/jpeg"
image_b64 = base64.b64encode(httpx.get(image_url).content).decode("utf-8")

response = requests.get(image_url)
image = Image(response.content, width=300, height=200)

image

单次调用

为Anthropic的Vertex AI初始化一个客户端，发送一个请求来描述一张图像（以base64编码的数据提供），以及一个文本提示，并以格式化的JSON打印响应。

In [ ]:
from anthropic import AnthropicVertex

client = AnthropicVertex(region=LOCATION, project_id=PROJECT_ID)

message = client.messages.create(
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": image_media_type,
                        "data": image_b64,
                    },
                },
                {"type": "text", "text": "Describe this image."},
            ],
        }
    ],
    model=MODEL,
)
print(message.model_dump_json(indent=2))

实时调用

初始化Anthropic的Vertex AI客户端，发送一个流式请求来描述一张图像（以base64编码数据提供），并连续打印接收到的文本作为流式传输。

In [ ]:
from anthropic import AnthropicVertex

client = AnthropicVertex(region=LOCATION, project_id=PROJECT_ID)

with client.messages.stream(
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": image_media_type,
                        "data": image_b64,
                    },
                },
                {"type": "text", "text": "Describe this image."},
            ],
        }
    ],
    model=MODEL,
) as stream:
    for text in stream.text_stream:
        print(text, end="", flush=True)